# Шаг 2: Сканирование приложений с помощью Quark Engine и составление отчётов об уязвимостях

In [1]:
import os
import re
import csv
import pandas as pd
import json
from interruptingcow import timeout
import signal
from quark.script import runQuarkAnalysis, Rule, findMethodInAPK, getApplication, checkMethodCalls, getReceivers, getActivities

In [2]:
APK_PATH = "D:\\ETU\\VKR\\apk_all"
RULE_MAIN_PATH = "D:\\ETU\\VKR\\rules"
JSON_PATH = "D:\\ETU\\VKR\\json\\"

In [3]:
def write_to_json(cwe_id="", description="", caller="", target="", json_obj={}):
    path = caller.split(";")[0][1:].replace("/", "\\")
    caller_method = caller.split(";", 1)[1].replace(" ", "")
    target_method = target.split(";")[1].replace(" ", "")

    cwe = json_obj["cwe"]
    
    new_cwe = {
        "id":cwe_id,
        "description":description,
        "caller":caller_method,
        "target":target_method
    }
    
    if not cwe:
        new_list_entry = {
        "path":path,
        "list":[]
        }
        new_list_entry["list"].append(new_cwe)
        json_obj["cwe"].append(new_list_entry)
    else:
        entry_path = ""
        for entry in cwe:
            if path == entry["path"]:
                entry_path = entry["path"]
        
        if entry_path:
            for entry in cwe:
                if entry["path"] == entry_path: entry["list"].append(new_cwe)
        else:
            new_list_entry = {
                "path":path,
                "list":[]
            }
            new_list_entry["list"].append(new_cwe)
            json_obj["cwe"].append(new_list_entry)

## Функции поиска уязвимостей

In [4]:
def CWE_22(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "accessFileInExternalDir.json"

    STRING_MATCHING_API = [
        ["Ljava/lang/String;", "contains", "(Ljava/lang/CharSequence)Z"],
        ["Ljava/lang/String;", "indexOf", "(I)I"],
        ["Ljava/lang/String;", "indexOf", "(Ljava/lang/String;)I"],
        ["Ljava/lang/String;", "matches", "(Ljava/lang/String;)Z"],
    ]


    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)
    
    for accessExternalDir in quarkResult.behaviorOccurList:
        target =  accessExternalDir.firstAPI.fullName.split(";")[1].replace(" ", "")
        filePath = [s for s in accessExternalDir.secondAPI.getArguments() if target in s][0]
        
        if quarkResult.isHardcoded(filePath):
            continue

        caller = accessExternalDir.methodCaller
        strMatchingAPIs = [
                api for api in STRING_MATCHING_API if quarkResult.findMethodInCaller(
                    caller, api)
        ]

        if not strMatchingAPIs:
            print(f"CWE-22 is detected in method, {caller.fullName}")
            write_to_json(
                cwe_id="CWE-22",
                description="Improper Limitation of a Pathname to a Restricted Directory",
                caller=caller.fullName,
                target=accessExternalDir.firstAPI.fullName,
                json_obj=json_obj)

In [5]:
def CWE_23(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "accessFileInExternalDir.json"

    STRING_MATCHING_API = [
        ["Ljava/lang/String;", "contains", "(Ljava/lang/CharSequence)Z"],
        ["Ljava/lang/String;", "indexOf", "(I)I"],
        ["Ljava/lang/String;", "indexOf", "(Ljava/lang/String;)I"],
        ["Ljava/lang/String;", "matches", "(Ljava/lang/String;)Z"],
        ["Ljava/lang/String;", "replaceAll",
            "(Ljava/lang/String; Ljava/lang/String;)Ljava/lang/String;"],
    ]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for accessExternalDir in quarkResult.behaviorOccurList:

        filePath = accessExternalDir.secondAPI.getArguments()[2]

        if quarkResult.isHardcoded(filePath):
            continue

        caller = accessExternalDir.methodCaller
        strMatchingAPIs = [
            api for api in STRING_MATCHING_API if quarkResult.findMethodInCaller(
                caller, api)
        ]

        if not strMatchingAPIs:
            print(f"CWE-23 is detected in method, {caller.fullName}")
            write_to_json(
                cwe_id="CWE-23", 
                description="Relative Path Traversal",
                caller=caller.fullName,
                target=accessExternalDir.firstAPI.fullName,
                json_obj=json_obj)
        elif strMatchingAPIs.find("..") == -1:
            print(f"CWE-23 is detected in method, {caller.fullName}")
            write_to_json(
                cwe_id="CWE-23", 
                description="Relative Path Traversal",
                caller=caller.fullName,
                target=accessExternalDir.firstAPI.fullName,
                json_obj=json_obj)

In [6]:
def CWE_73(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "accessFileInExternalDir.json"

    OPEN_FILE_API = [
        "Landroid/os/ParcelFileDescriptor;",                   # Class name
        "open",                                                # Method name
        "(Ljava/io/File; I)Landroid/os/ParcelFileDescriptor;"  # Descriptor
    ]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for accessExternalDir in quarkResult.behaviorOccurList:
        filePath = accessExternalDir.secondAPI.getArguments()[2]

        if quarkResult.isHardcoded(filePath):
            continue

        caller = accessExternalDir.methodCaller
        result = quarkResult.findMethodInCaller(caller, OPEN_FILE_API)

        if result:
            print("CWE-73 is detected in method, ", caller.fullName)
            write_to_json(
                cwe_id="CWE-73",
                description="External Control of File Name or Path",
                caller=caller.fullName,
                target=accessExternalDir.firstAPI.fullName,
                json_obj=json_obj)

In [7]:
def CWE_78(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "ExternalStringCommand.json"

    STRING_MATCHING_API = set([
        ("Ljava/lang/String;", "contains", "(Ljava/lang/CharSequence)Z"),
        ("Ljava/lang/String;", "indexOf", "(I)I"),
        ("Ljava/lang/String;", "indexOf", "(Ljava/lang/String;)I"),
        ("Ljava/lang/String;", "matches", "(Ljava/lang/String;)Z"),
        ("Ljava/lang/String;", "replaceAll", "(Ljava/lang/String; Ljava/lang/String;)Ljava/lang/String;")
    ])

    specialElementsPattern = r"[ ;|,>`]+"

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for ExternalStringCommand in quarkResult.behaviorOccurList:

        methodCalled = set()
        caller = ExternalStringCommand.methodCaller

        for method in ExternalStringCommand.getMethodsInArgs():
            methodCalled.add(method.fullName)

        if methodCalled.intersection(STRING_MATCHING_API) and not ExternalStringCommand.hasString(specialElementsPattern):
            continue
        else:
            print(f"CWE-78 is detected in method, {caller.fullName}")
            write_to_json(
                cwe_id="CWE-78",
                description="Improper Neutralization of Special Elements used in an OS Command",
                caller=caller.fullName,
                target=ExternalStringCommand.firstAPI.fullName,
                json_obj=json_obj)

In [9]:
def CWE_88(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "ExternalStringCommand.json"


    STRING_MATCHING_API = set([
        ("Ljava/lang/String;", "contains", "(Ljava/lang/CharSequence)Z"),
        ("Ljava/lang/String;", "indexOf", "(I)I"),
        ("Ljava/lang/String;", "indexOf", "(Ljava/lang/String;)I"),
        ("Ljava/lang/String;", "matches", "(Ljava/lang/String;)Z"),
        ("Ljava/lang/String;", "replaceAll", "(Ljava/lang/String; Ljava/lang/String;)Ljava/lang/String;")
    ])

    delimeter = "-"

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for ExternalStringCommand in quarkResult.behaviorOccurList:

        methodCalled = set()
        caller = ExternalStringCommand.methodCaller

        for method in ExternalStringCommand.getMethodsInArgs():
            methodCalled.add(method.fullName)

        if methodCalled.intersection(STRING_MATCHING_API) and not ExternalStringCommand.hasString(delimeter):
            continue
        else:
            print(f"CWE-88 is detected in method, {caller.fullName}")
            
            write_to_json(
                cwe_id="CWE-88",
                description="Improper Neutralization of Argument Delimiters in a Command",
                caller=caller.fullName,
                target=ExternalStringCommand.firstAPI.fullName,
                json_obj=json_obj)

In [10]:
def CWE_89(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "executeSQLCommand.json"

    targetMethod = [
        "Landroid/widget/EditText;", # class name
        "getText",                   # method name
        "()Landroid/text/Editable;", # descriptor
    ]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for sqlCommandExecution in quarkResult.behaviorOccurList:
        if sqlCommandExecution.isArgFromMethod(
            targetMethod
        ):
            print(f"CWE-89 is detected in {SAMPLE_PATH}")
            
            write_to_json(
                cwe_id="CWE-89",
                description="Improper Neutralization of Special Elements used in an SQL Command",
                caller=sqlCommandExecution.methodCaller.fullName,
                target=sqlCommandExecution.secondAPI.fullName,
                json_obj=json_obj)

In [11]:
#Тут тоже посмотреть, какой API брать
def CWE_94(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "loadExternalCode.json"

    targetMethod = [
            "Landroid/content/pm/PackageManager;",
            "checkSignatures",
            "(Ljava/lang/String;Ljava/lang/String;)I"
            ]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for ldExternalCode in quarkResult.behaviorOccurList:

        callerMethod = [
                ldExternalCode.methodCaller.className,
                ldExternalCode.methodCaller.methodName,
                ldExternalCode.methodCaller.descriptor
                ]

        if not quarkResult.findMethodInCaller(callerMethod, targetMethod):
            print(f"Method: {targetMethod[1]} not found! CWE-94 is detected in {ldExternalCode.methodCaller.fullName}")
            
            write_to_json(
                cwe_id="CWE-94",
                description="Improper Control of Generation of Code",
                caller=ldExternalCode.methodCaller.fullName,
                target=ldExternalCode.secondAPI.fullName, # 2 loadClass / 1 createPackageContext
                json_obj=json_obj)

In [12]:
def CWE_117(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "writeContentToLog.json"
    KEYWORDS_FOR_NEUTRALIZATION = ["escape", "replace", "format", "setFilter"]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for logOutputBehavior in quarkResult.behaviorOccurList:

        secondAPIParam = logOutputBehavior.getParamValues()[1]

        isKeywordFound = False
        for keyword in KEYWORDS_FOR_NEUTRALIZATION:
            if keyword in secondAPIParam:
                isKeywordFound = True
                break

        if not isKeywordFound:
            print(f"CWE-117 is detected in method,{secondAPIParam}")
            
            write_to_json(
                cwe_id="CWE-117",
                description="Improper Output Neutralization for Logs",
                caller=secondAPIParam,
                target=logOutputBehavior.secondAPI.fullName, # 2 d()Landroid/util/Log // 1 getText
                json_obj=json_obj)

In [13]:
def CWE_295(SAMPLE_PATH, json_obj):
    TARGET_METHOD = [
        "Landroid/webkit/SslErrorHandler;",  # class name
        "proceed",                           # method name
        "()V"                                # descriptor
    ]
    OVERRIDE_METHOD = [
        "Landroid/webkit/WebViewClient;",    # class name
        "onReceivedSslError",                # method name
        "(Landroid/webkit/WebView;"+" Landroid/webkit/SslErrorHandler;" + \
        " Landroid/net/http/SslError;)V"     # descriptor
    ]

    for sslProceedCaller in findMethodInAPK(SAMPLE_PATH, TARGET_METHOD):
        if (sslProceedCaller.name == OVERRIDE_METHOD[1] and
           sslProceedCaller.descriptor == OVERRIDE_METHOD[2] and
           OVERRIDE_METHOD[0] in sslProceedCaller.findSuperclassHierarchy()):
            print(f"CWE-295 is detected in method, {sslProceedCaller.fullName}")
            
            write_to_json(
                cwe_id="CWE-295",
                description="Improper Certificate Validation",
                caller=sslProceedCaller.fullName,
                target=" ".join(str(e) for e in OVERRIDE_METHOD),
                json_obj=json_obj)

In [14]:
def CWE_319(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "setRetrofitBaseUrl.json"

    PROTOCOL_KEYWORDS = [
        "http",
        "smtp",
        "ftp"
    ]


    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for setRetrofitBaseUrl in quarkResult.behaviorOccurList:
        for protocol in PROTOCOL_KEYWORDS:

            regexRule = f"{protocol}://[0-9A-Za-z./-]+"
            cleartextProtocolUrl = setRetrofitBaseUrl.hasString(regexRule, True)

            if cleartextProtocolUrl:
                print(f"CWE-319 detected!")
                print(f"Here are the found URLs with cleartext protocol:")
                print("\n".join(cleartextProtocolUrl))
                
                write_to_json(
                    cwe_id="CWE-319",
                    description="Cleartext Transmission of Sensitive Information",
                    caller=setRetrofitBaseUrl.methodCaller.fullName,
                    target=setRetrofitBaseUrl.secondAPI.fullName, # 2 baseUrl // <init>
                    json_obj=json_obj)

In [15]:
def CWE_327(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "useOfCryptographicAlgo.json"

    WEAK_ALGORITHMS = ["DES", "ARC4", "BLOWFISH"]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for useCryptoAlgo in quarkResult.behaviorOccurList:

        caller = useCryptoAlgo.methodCaller

        for algo in WEAK_ALGORITHMS:
            if useCryptoAlgo.hasString(algo):
                print(f"CWE-327 is detected in method, {caller.fullName}")
                
                write_to_json(
                    cwe_id="CWE-327",
                    description="Use of a Broken or Risky Cryptographic Algorithm",
                    caller=caller.fullName,
                    target=useCryptoAlgo.firstAPI.fullName, # 1 getInstance / 2 init
                    json_obj=json_obj)

In [16]:
def CWE_328(SAMPLE_PATH, json_obj):
    TARGET_METHODS = [
        [
            "Ljava/security/MessageDigest;", "getInstance",
            "(Ljava/lang/String;)Ljava/security/MessageDigest;"
        ],
        [
            "Ljavax/crypto/SecretKeyFactory;", "getInstance",
            "(Ljava/lang/String;)Ljavax/crypto/SecretKeyFactory;"
        ]
    ]

    HASH_KEYWORDS = [
        "MD2",  "MD4",  "MD5",      "PANAMA",
        "SHA0", "SHA1", "HAVAL128", "RIPEMD128"
    ]

    #for samplePath in SAMPLE_PATH:
        #print("cwe 328 path: ", samplePath)
    methodsFound = []
    for target in TARGET_METHODS:
        #methodsFound += findMethodInAPK(samplePath, target)
        methodsFound += findMethodInAPK(SAMPLE_PATH, target)

    for setHashAlgo in methodsFound:
        algoName = setHashAlgo.getArguments()[0].replace("-", "")

        if any(keyword in algoName for keyword in HASH_KEYWORDS):
            print(f"CWE-328 is detected in {SAMPLE_PATH},\n\t"
                  f"and it occurs in method, {setHashAlgo.fullName}")
            
            write_to_json(
                    cwe_id="CWE-328",
                    description="Use of Weak Hash",
                    caller=setHashAlgo.fullName,
                    target=useCryptoAlgo.firstAPI.fullName, # 1 getInstance / 2 init
                    json_obj=json_obj)

In [17]:
#А это тоже не использовать?
def CWE_338(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "useMethodOfPRNG.json"

    CREDENTIAL_KEYWORDS = [
        "token", "password", "account", "encrypt",
        "authentication", "authorization", "id", "key"
    ]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for usePRNGMethod in quarkResult.behaviorOccurList:
        for prngCaller in usePRNGMethod.methodCaller.getXrefFrom():
            if any(keyword in prngCaller.fullName
                   for keyword in CREDENTIAL_KEYWORDS):
                print("CWE-338 is detected in %s" % prngCaller.fullName)
                
                write_to_json(
                    cwe_id="CWE-338",
                    description="Use of Cryptographically Weak Pseudo-Random Number Generator",
                    caller=prngCaller.fullName,
                    target=usePRNGMethod.secondAPI.fullName, # 2 nextInt / 1 <init>
                    json_obj=json_obj)

In [18]:
#Как вписать?
def CWE_489(SAMPLE_PATH, json_obj):
    if getApplication(SAMPLE_PATH).isDebuggable():
        print(f"CWE-489 is detected in {SAMPLE_PATH}.")
        
        #append_row(SAMPLE_PATH, "CWE-489", "Active Debug Code")

In [19]:
def CWE_502(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "deserializeData.json"

    ruleInstance = Rule(RULE_PATH)

    result = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    verificationApis = [
        ["Ljava/io/File;", "exists", "()Z"],
        ["Landroid/content/Context;", "getFilesDir", "()Ljava/io/File;"],
        ["Landroid/content/Context;", "getExternalFilesDir", "(Ljava/lang/String;)Ljava/io/File;"],
        ["Landroid/os/Environment;", "getExternalStorageDirectory", "()Ljava/io/File;"],
    ]

    for dataDeserialization in result.behaviorOccurList:
        apis = dataDeserialization.getMethodsInArgs()
        caller = dataDeserialization.methodCaller
        if not any(api in apis for api in verificationApis):
            print(f"CWE-502 is detected in method, {caller.fullName}")
            
            write_to_json(
                cwe_id="CWE-502",
                description="Deserialization of Untrusted Data",
                caller=dataDeserialization.methodCaller.fullName,
                target=dataDeserialization.secondAPI.fullName, # 2 readObject / 1 <init>
                json_obj=json_obj)

In [20]:
def CWE_532(SAMPLE_PATH, json_obj):
    TARGET_METHOD = [
        "Landroid/util/Log;",                       # class name
        "d",                                        # method name
        "(Ljava/lang/String; Ljava/lang/String;)I"  # descriptor
    ]
    CREDENTIAL_KEYWORDS = [
        "token",
        "decrypt",
        "password"
    ]

    methodsFound = findMethodInAPK(SAMPLE_PATH, TARGET_METHOD)

    for debugLogger in methodsFound:
        arguments = debugLogger.getArguments()
        
        if len(arguments) > 1:
            for keyword in CREDENTIAL_KEYWORDS:
                #if keyword in arguments[1]:
                if keyword in arguments[0]:
                    print(f"CWE-532 is detected in method, {debugLogger.fullName}")

                    write_to_json(
                        cwe_id="CWE-532",
                        description="Insertion of Sensitive Information into Log File",
                        caller=debugLogger.fullName,
                        target=" ".join(str(e) for e in TARGET_METHOD),
                        json_obj=json_obj)

In [21]:
#Проверить таргет на примере
def CWE_601(SAMPLE_PATH, json_obj):
    TARGET_METHOD = ["", "startActivity", "(Landroid/content/Intent;)V"]  
    target = ";" + TARGET_METHOD[1] + TARGET_METHOD[2]
    """
    Due to varying descriptors and classes in smali code from different APIs, 
    our search relies solely on the consistent method names.
    """

    EXTERNAL_INPUT_METHODS = [
       "getIntent", 
       "getQueryParameter"
    ]

    INPUT_FILTER_METHODS = [
       "parse", 
       "isValidUrl", 
       "Pattern", 
       "Matcher", 
       "encode", 
       "decode", 
       "escapeHtml", 
       "HttpURLConnection"
    ]

    redirectMethods = findMethodInAPK(SAMPLE_PATH, TARGET_METHOD)

    for redirectMethod in redirectMethods:
        arguments = redirectMethod.getArguments()
        for argument in arguments:
            if any(externalInput in argument for externalInput in EXTERNAL_INPUT_METHODS):
                if not any(filterMethod in argument for filterMethod in INPUT_FILTER_METHODS):
                    print(f"CWE-601 is detected in {redirectMethod.fullName}")

                    write_to_json(
                        cwe_id="CWE-601",
                        description="URL Redirection to Untrusted Site",
                        caller=redirectMethod.fullName,
                        target=target,
                        json_obj=json_obj)

In [22]:
def CWE_749(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "configureJsExecution.json"

    targetMethod = [
        "Landroid/webkit/WebView;",
        "addJavascriptInterface",
        "(Ljava/lang/Object; Ljava/lang/String;)V"
    ]

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for configureJsExecution in quarkResult.behaviorOccurList:
        caller = configureJsExecution.methodCaller
        secondAPI = configureJsExecution.secondAPI

        if len(secondAPI.getArguments()) > 0:
            #enableJS = secondAPI.getArguments()[1]
            enableJS = secondAPI.getArguments()[0]
            exposeAPI = quarkResult.findMethodInCaller(caller, targetMethod)

            if enableJS and exposeAPI:
                print(f"CWE-749 is detected in method, {caller.fullName}")
                
                write_to_json(
                    cwe_id="CWE-749",
                    description="Exposed Dangerous Method or Function",
                    caller=configureJsExecution.methodCaller.fullName,
                    target=configureJsExecution.secondAPI.fullName,
                    json_obj=json_obj)

In [23]:
def CWE_780(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "useOfCryptographicAlgo.json"

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for useCryptographicAlgo in quarkResult.behaviorOccurList:

        methodCaller = useCryptographicAlgo.methodCaller

        if useCryptographicAlgo.hasString("RSA") and \
            not useCryptographicAlgo.hasString("OAEP"):
            print(f"CWE-780 is detected in method, {methodCaller.fullName}")
            
            write_to_json(
                cwe_id="CWE-780",
                description="Use of RSA Algorithm without OAEP",
                caller=methodCaller.fullName,
                target=useCryptographicAlgo.firstAPI.fullName, # 1 getInstance / 2 init
                json_obj=json_obj)

In [24]:
def CWE_798(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "findSecretKeySpec.json"

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for secretKeySpec in quarkResult.behaviorOccurList:

        allStrings = quarkResult.getAllStrings()

        firstParam = secretKeySpec.getParamValues()[1]
        secondParam = secretKeySpec.getParamValues()[2]

        if secondParam == "AES":
            AESKey = re.findall(r'\((.*?)\)', firstParam)[1]

        if AESKey in allStrings:
            print(f"Found hard-coded {secondParam} key {AESKey}")
            
            write_to_json(
                cwe_id="CWE-798",
                description="Use of Hard-coded Credentials",
                caller=secretKeySpec.methodCaller.fullName,
                target=secretKeySpec.firstAPI.fullName, # 1 getBytes / 2 init
                json_obj=json_obj)

In [25]:
def CWE_921(SAMPLE_PATH, json_obj):
    RULE_PATH = RULE_MAIN_PATH + "/" + "checkFileExistence.json"

    ruleInstance = Rule(RULE_PATH)
    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for existingFile in quarkResult.behaviorOccurList:
        filePath = existingFile.getParamValues()[0]
        if "sdcard" in filePath:
            print(f"This file is stored inside the SDcard\n")
            print(f"CWE-921 is detected in {existingFile.methodCaller.fullName}.")
            
            write_to_json(
                cwe_id="CWE-921",
                description="Storage of Sensitive Data in a Mechanism without Access Control",
                caller=existingFile.methodCaller.fullName,
                target=existingFile.secondAPI.fullName, # 2 exists / 1 init
                json_obj=json_obj)

In [26]:
def CWE_925(SAMPLE_PATH, json_obj):
    TARGET_METHOD = [
        '',
        'onReceive',
        '(Landroid/content/Context; Landroid/content/Intent;)V'
    ]

    CHECK_METHODS = [
        ['Landroid/content/Intent;', 'getAction', '()Ljava/lang/String;']
    ]

    receivers = getReceivers(SAMPLE_PATH)
    for receiver in receivers:
        if receiver.isExported():
            className = "L"+str(receiver).replace('.', '/')+';'
            TARGET_METHOD[0] = className
            if not checkMethodCalls(SAMPLE_PATH, TARGET_METHOD, CHECK_METHODS):
                print(f"CWE-925 is detected in method, {className}")
                
                write_to_json(
                    cwe_id="CWE-925",
                    description="Improper Verification of Intent by Broadcast Receiver",
                    caller=className,
                    target=TARGET_METHOD[1] + TARGET_METHOD[2],
                    json_obj=json_obj)

In [27]:
def CWE_926(SAMPLE_PATH, json_obj):
    for activityInstance in getActivities(SAMPLE_PATH):

        if activityInstance.hasIntentFilter() and activityInstance.isExported():
            print(f"CWE-926 is detected in the activity, {activityInstance}")

In [28]:
def CWE_940(SAMPLE_PATH, json_obj):
    RULE_PATH =  RULE_MAIN_PATH + "/" +  "LoadUrlFromIntent.json"

    INTENT_SETTING_METHODS = [
        "findViewById",
        "getStringExtra",
        "getIntent",
    ]

    ruleInstance = Rule(RULE_PATH)

    quarkResult = runQuarkAnalysis(SAMPLE_PATH, ruleInstance)

    for behaviorInstance in quarkResult.behaviorOccurList:
        methodsInArgs = behaviorInstance.getMethodsInArgs()

        verifiedMethodCandidates = []

        for method in methodsInArgs:
            if method.methodName not in INTENT_SETTING_METHODS:
                verifiedMethodCandidates.append(method)

        if verifiedMethodCandidates == []:
            caller = behaviorInstance.methodCaller.fullName
            print(f"cwe-940 is detected in method, {caller}")
            
            write_to_json(
                cwe_id="CWE-940",
                description="Improper Verification of Source of a Communication Channel",
                caller=caller,
                target=behaviorInstance.secondAPI.fullName, # 2 loadUrl / 1 getData
                json_obj=json_obj)

In [ ]:
files = os.listdir(APK_PATH)
for file in files:
    try:
        current_path = APK_PATH + "/" + file
        app = json.dumps({"cwe":[]})
        app_json = json.loads(app)

        print("***************File Name : "+file+"***************")

        CWE_22(current_path, app_json)
        CWE_79(current_path, app_json)
        CWE_88(current_path, app_json)
        CWE_89(current_path, app_json)
        CWE_117(current_path, app_json)
        CWE_319(current_path, app_json)
        CWE_502(current_path, app_json)
        CWE_532(current_path, app_json)
        CWE_601(current_path, app_json)
        CWE_749(current_path, app_json)
        CWE_921(current_path, app_json)
    except Exception as e:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(e)
        continue
    with open(JSON_PATH + file + '.json', 'w') as outfile:
        json.dump(app_json, outfile)

***************File Name : InjuredAndroid.apk***************


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


CWE-502 is detected in method, Lio/flutter/plugins/a/a; c (Ljava/lang/String;)Ljava/util/List;
***************File Name : InsecureShop.apk***************


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


CWE-79 is detected in method, Lcom/insecureshop/WebViewActivity; onCreate (Landroid/os/Bundle;)V
CWE-79 is detected in method, Lcom/insecureshop/WebView2Activity; onCreate (Landroid/os/Bundle;)V
CWE-117 is detected in method,Ljava/lang/String;->valueOf(Ljava/lang/Object;)Ljava/lang/String;(Lcom/google/android/material/textfield/TextInputEditText;->getText()Landroid/text/Editable;(Lcom/google/android/material/textfield/TextInputEditText;->getText()Landroid/text/Editable;(view)))
CWE-502 is detected in method, Landroidx/versionedparcelable/VersionedParcel; readSerializable ()Ljava/io/Serializable;
CWE-601 is detected in Lcom/insecureshop/WebView2Activity; onCreate (Landroid/os/Bundle;)V
***************File Name : MSTG-Android-Java.apk***************
CWE-22 is detected in method, Lsg/vp/owasp_mobile/OMTG_Android/OMTG_DATAST_001_ExternalStorage; onCreate (Landroid/os/Bundle;)V
CWE-502 is detected in method, Landroidx/versionedparcelable/VersionedParcel; readSerializable ()Ljava/io/Serializ

...

Requested API level 29 is larger than maximum we have, returning API level 28 instead.


CWE-502 is detected in method, Landroidx/work/Data; fromByteArray ([B)Landroidx/work/Data;
***************File Name : fediphotolineage.apk***************


Requested API level 34 is larger than maximum we have, returning API level 28 instead.


CWE-502 is detected in method, Le2/g; a ([B)Le2/g;
***************File Name : feeder.apk***************


Requested API level 34 is larger than maximum we have, returning API level 28 instead.
